In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('/content/Train.csv')

In [3]:
train.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,6141,1583,144,3,2011-05-06 16:54:00,3.75,14056.0,35
1,6349,1300,3682,6,2011-05-11 07:35:00,1.95,13098.0,35
2,16783,2178,1939,4,2011-11-20 13:20:00,5.95,15044.0,35
3,16971,2115,2983,1,2011-11-22 12:07:00,0.83,15525.0,35
4,6080,1210,2886,12,2011-05-06 09:00:00,1.65,13952.0,35


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284780 entries, 0 to 284779
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    284780 non-null  int64  
 1   StockCode    284780 non-null  int64  
 2   Description  284780 non-null  int64  
 3   Quantity     284780 non-null  int64  
 4   InvoiceDate  284780 non-null  object 
 5   UnitPrice    284780 non-null  float64
 6   CustomerID   284780 non-null  float64
 7   Country      284780 non-null  int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 17.4+ MB


In [5]:
!pip install h2o
import h2o
from h2o.automl import H2OAutoML
h2o.init()

     |████████████████████████████████| 129.4MB 75kB/s 
  Created wheel for h2o: filename=h2o-3.30.1.3-py2.py3-none-any.whl size=129446676 sha256=ad1a282c15801112d2a36beed5d9bf896755a8090cd8418fe5ce31cfa4b8e576
  Stored in directory: /root/.cache/pip/wheels/3d/e6/07/53ce9be9cb61b33a79cb3ed1fc39f2dae84f6ee6fe1e373e5e
Successfully built h2o
Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.9" 2020-10-20; OpenJDK Runtime Environment (build 11.0.9+11-Ubuntu-0ubuntu1.18.04.1); OpenJDK 64-Bit Server VM (build 11.0.9+11-Ubuntu-0ubuntu1.18.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmps5fnvb4w
  JVM stdout: /tmp/tmps5fnvb4w/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmps5fnvb4w/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connectin

H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.3
H2O_cluster_version_age:,1 month and 12 days
H2O_cluster_name:,H2O_from_python_unknownUser_45s4m1
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [6]:
data = h2o.import_file('/content/Train.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [7]:
features=data.columns
features.remove('UnitPrice')
features.remove('CustomerID')
features.remove('InvoiceDate')
y='UnitPrice'
aml = H2OAutoML(max_models=10,seed=1,nfolds=5,verbosity='warn')
aml.train(x=features,y='UnitPrice',training_frame = data)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [8]:
aml.leaderboard

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_5_AutoML_20201110_121714,5976.27,77.3064,5976.27,1.59343,nan
StackedEnsemble_BestOfFamily_AutoML_20201110_121714,6034.05,77.6792,6034.05,1.57832,nan
StackedEnsemble_AllModels_AutoML_20201110_121714,6049.98,77.7816,6049.98,1.5843,nan
GLM_1_AutoML_20201110_121714,6146.37,78.3988,6146.37,2.84731,0.693768
GBM_3_AutoML_20201110_121714,6183.67,78.6363,6183.67,2.21015,nan
DRF_1_AutoML_20201110_121714,6214.17,78.83,6214.17,2.30767,0.522553
GBM_2_AutoML_20201110_121714,6222.07,78.8801,6222.07,2.27024,nan
XGBoost_2_AutoML_20201110_121714,6244.49,79.0221,6244.49,1.53764,nan
GBM_4_AutoML_20201110_121714,6247.7,79.0424,6247.7,1.93111,nan
XGBoost_1_AutoML_20201110_121714,6818.87,82.5764,6818.87,2.11614,nan


In [9]:
from sklearn.metrics import mean_squared_error
test = h2o.import_file('/content/Test.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [10]:
predictions = aml.predict(test)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [11]:
submit = pd.DataFrame(predictions.as_data_frame(use_pandas=True)['predict'].tolist())

In [12]:
submit.columns=['UnitPrice']
submit

,UnitPrice
0,1.852142
1,1.575618
2,2.933390
3,1.306669
4,11.193170
...,...
122044,0.762008
122045,1.592646
122046,1.210586
122047,5.221494


In [ ]:
from google.colab import files
submit.to_csv('HP1.csv')         
files.download('HP1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>